# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [7]:
len(data)

4870

## 1. What test is appropriate for this problem? Does CLT apply?

A two sample binomial test would be appropriate since we have two binomial proportions. The CLT does apply here and we can use the large sample approximation aka the two sample proportion z test because we have more than enough data points (4870).

## 2. What are the null and alternate hypotheses?

The null hypothesis is that p2 - p1 = 0 where p1 = Proportion of black-sounding resumes receiving a call from employers and p2 = Proportion of white-sounding resumes receiving a call from employers.

The alternative hypothesis is that p2 - p1 =/= 0

## 3. Compute margin of error, confidence interval, and p-value.

In [10]:
sample_p1 = data.query('race == "b"')['call'].mean()
sample_p1

0.064476386

In [11]:
sample_p2 = data.query('race == "w"')['call'].mean()
sample_p2

0.09650924

In [15]:
sample_difference = sample_p2 - sample_p1
sample_difference

0.032032855

In [12]:
len(data.query('race == "b"'))

2435

In [13]:
len(data.query('race == "w"'))

2435

Using alpha = 0.05,

Z(1-alpha/2) = 1.96


In [14]:
margin_of_error = 1.96 * np.sqrt(sample_p1*(1-sample_p1)/2435 + sample_p2*(1-sample_p2)/2435)
margin_of_error

0.015255406348684322

In [17]:
CI95 = (sample_difference - margin_of_error, sample_difference + margin_of_error)
CI95

(0.016777448506376254, 0.0472882612037449)

In [21]:
sample_pooled_prop = sum(data['call'])/4870

In [22]:
SE_prop = np.sqrt(2*sample_pooled_prop*(1-sample_pooled_prop)/2435)
SE_prop

0.007796894036170457

In [24]:
z_statistic = sample_difference / SE_prop
z_statistic

4.108412235238472

Using a normal table, the p value is 2*P(z > 4.11) < 0.004

Since 0.004 < 0.05, we reject the null hypothesis that p2 - p1 = 0

## 4. Write a story describing the statistical significance in the context or the original problem.

There is sufficient evidence to conclude that the proportion of the black-sounding resumes receiving calls from employers is not equal to the proportion of the white-sounding resumes receiving calls from employers based on the hypothesis test. There is less than 0.4% chance that the statistical significance happened by chance. This means that if you have a white-sounding name with the same credentials as someone else with a black-sounding name, you are more likely to be selected for an interview.

## 5.Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

The above results does not mean that race/name is the most important factor in callback success. Based on the description of the dataset, I am not sure whether or not if each resume of equal credential was sent to the same employer. It would be helpful in a future study to pair each resume to every employer/job so that there is less confounding. For an example, if a black-sounding resume with 4 years of experience in chemistry was sent to an entry level chemist position while a white-sounding resume with 4 years of experience in chemistry was sent to an accounting position, the black-sounding resume would be more likely to receive a call from the employer. 

Assuming that the above already holds for the dataset, instead of a hypothesis test, I would conduct logistic regression. The response variable would be the 'call' column. I would include the 'race' as a possible predictor (coded as a dummy variable) and do a binary logistic regression first on each of the variables to determine which ones are significant. Then I would throw the significant ones all in one multiple logistic regression model and build my model from there. Depending on how large of the coefficients for the predictors are in the final model would determine how important each predictor/factor is in determining callback success